In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Lobby as LobbyModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

Models = [
    BracketModel, 
    LobbyModel,
    TournamentModel, 
    MatchModel, 
    RoundModel, 
    UserModel
]

for Model in Models:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [12]:
# log miguel in
url = 'http://127.0.0.1:5000/api/login'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"data":"Hello, miguel!"}\n'

In [9]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })
    r = requests.post(url, data=payload, headers=headers)


In [10]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [11]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzY5NjY0MCwiZXhwIjoxNTgzNjk3MjQwfQ.eyJpZCI6MX0.Xzr9JySdLqPndM92tcV4NHrV2JrNGpGRhL-T45VFXvaRPlXjxS_CSfLxxXdPNMiCNPW58Gor8Avhrh5-0MSSnw'

In [12]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [13]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourneys"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Lobby Created': "Lobby created for tournament: miguel's tourneys"}

In [14]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,to_id
0,1,miguel's tourneys,1


In [15]:
q = \
"""
    select * from user
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df.head(10)

,id,role,username,current_seed,email,password_hash,about_me,last_seen,lobby_id
0,1,User,miguel,None,miguel@gmail.com,pbkdf2:sha256:150000$8IznJZpS$8ca897f587bdedfb...,None,2020-03-08 19:43:59.241648,None
1,2,User,TPN,None,tpn@example.com,pbkdf2:sha256:150000$NtTtg2oK$2ff5cf8d77007936...,None,2020-03-08 19:43:59.347790,None
2,3,User,AngryFalco,None,angryfalco@example.com,pbkdf2:sha256:150000$KOyOW1A9$b4a2196853f1d06d...,None,2020-03-08 19:43:59.437481,None
3,4,User,Sunrisebanana,None,sunrisebanana@example.com,pbkdf2:sha256:150000$cLaDpppI$0490ff1d9ef1ad8e...,None,2020-03-08 19:43:59.522311,None
4,5,User,Ptolemy,None,ptolemy@example.com,pbkdf2:sha256:150000$lTDCPFpr$edd92b5d401b84da...,None,2020-03-08 19:43:59.611114,None
5,6,User,Vik,None,vik@example.com,pbkdf2:sha256:150000$m2poY2WQ$b40d6e74eff9089a...,None,2020-03-08 19:43:59.692873,None
6,7,User,Kevin,None,kevin@example.com,pbkdf2:sha256:150000$0sgAN8oQ$08f915459d9ad669...,None,2020-03-08 19:43:59.771955,None
7,8,User,Spaceghost,None,spaceghost@example.com,pbkdf2:sha256:150000$MD0pXHza$48f112dfa847ec7d...,None,2020-03-08 19:43:59.853274,None
8,9,User,Burnaby,None,burnaby@example.com,pbkdf2:sha256:150000$kVn6ABTh$2f28b201e0cd9166...,None,2020-03-08 19:43:59.935560,None


In [16]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzY5NjY0MCwiZXhwIjoxNTgzNjk3MjQwfQ.eyJpZCI6MX0.Xzr9JySdLqPndM92tcV4NHrV2JrNGpGRhL-T45VFXvaRPlXjxS_CSfLxxXdPNMiCNPW58Gor8Avhrh5-0MSSnw'

In [17]:
# users = df.username.values
# np.random.shuffle(users)
# import random

# # miguel adds users to his lobby

# url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# seeds = random.sample(range(1, len(users)+1), len(users)) 
# roles = ["User" for u in users]
# for u, s, r in zip(users, roles, seeds):
#     payload = json.dumps({
#         "username" : u,
#         "role" : s,
#         "seed" : r
#     })
#     r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
#     print(json.loads(r.content))

# url = 'http://127.0.0.1:5000/api/lobby/1'
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# # payload = json.dumps({
# #     "username" : "AngryFalco",
# #     "role" : "User"
# # })
# r = requests.get(url, headers=headers,)
# json.loads(r.content)

# # miguel adds a guest user

# url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# payload = json.dumps({
#     "username" : "guesty",
#     "role" : "Guest",
#     "seed" : 10
# })
# r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
# print(json.loads(r.content))

# # log miguel out
# url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# # TPN tries to add his own guests to miguel's lobby
# url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# payload = json.dumps({
#     "username" : "guestx",
#     "role" : "Guest",
#     "seed" : 1
# })
# r = requests.post(url, data=payload, headers=headers, auth=('TPN', 'tpn'))
# print(json.loads(r.content))

# UserModel \
#     .query \
#     .filter_by(username='guesty', role='Guest') \
#     .first() is None